In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

In [2]:
# 학습, 평가데이터 불러오기

train = pd.read_csv('X_train.csv', encoding = 'cp949')
test = pd.read_csv('X_test.csv', encoding = 'cp949')
target = pd.read_csv('y_train.csv', encoding = 'cp949').group

In [3]:
# 사이버 쇼핑, 점외 등 1개 밖에 없는 data를 가지고 있는 행 삭제 

train.drop(index = 578987, inplace = True)

In [4]:
# 같이 전처리 하기 위해서 train과 test를 합침

data = pd.concat([train, test],ignore_index= True)

In [6]:
unique_custid = pd.DataFrame(data.custid.unique()).rename(columns = {0 : 'custid'})

# Preprocessing

In [7]:
# 시간데이터 처리

def hour_process(x):
    x = str(x)
    if len(x) == 4:
        return x[:2]
    elif len(x) == 3:
        return x[:1]
    elif len(x) == 2:
        return '0'

data['sales_hour'] = data['sales_time'].apply(hour_process)

def minute_process(x):
    x = str(x)
    if len(x) == 4:
        return x[2:]
    elif len(x) == 3:
        return x[1:]
    elif len(x) == 2:
        return x
    
data['sales_minute'] = data['sales_time'].apply(minute_process)


data['sales_hour'] = data['sales_hour'].astype(int)
data['sales_minute'] = data['sales_minute'].astype(int)

In [8]:
# 날짜데이터 처리

data['sales_month_origin'] = data['sales_month']
data['sales_month']=data['sales_month'].apply(lambda x: x-12 if x >12 else x)
data['year'] = data['sales_month_origin'].apply(lambda x: 2018 if x>12 else 2017)

data['year_month_day'] = data['year'].astype(str) + '_' + data['sales_month'].astype(str) + '_' +\
                                data['sales_day'].astype(str) + '_' + data['sales_time'].astype(str)

data['datetime'] = pd.to_datetime(data['year_month_day'], format = '%Y_%m_%d_%H%M')


data['diff_time'] = data.groupby('custid')['datetime'].diff().fillna('00:00:00').astype(str)
data['diff_time'] = data['diff_time'].str.split(' days').apply(lambda x: 0 if x[0] == '00:00:00' else x[0]).astype(int)

In [9]:
# corner_nm 에서 겹치는 값들 처리하기

data.loc[data.corner_nm == '모피.피혁', 'corner_nm'] = '모피/피혁'
data.loc[data.corner_nm == '원목(주니어)', 'corner_nm'] = '원목/주니어'
data.loc[data.corner_nm == '우산,장갑', 'corner_nm'] = '우산/장갑'
data.loc[data.corner_nm == '우산장갑', 'corner_nm'] = '우산/장갑'
data.loc[data.corner_nm == '트.단품 ', 'corner_nm'] = '트단품'
data.loc[data.corner_nm == 'TV,VTR', 'corner_nm'] = 'TV/VTR'
data.loc[data.corner_nm == 'TV.VTR', 'corner_nm'] = 'TV/VTR'
data.loc[data.corner_nm == 'GBR  지원', 'corner_nm'] = 'GBR지원'
data.loc[data.corner_nm == '페레  지원', 'corner_nm'] = '페레지원'
data.loc[data.corner_nm == '라디오.카세트', 'corner_nm'] = '라디오/카세트'
data.loc[data.corner_nm == '스포츠용퓸', 'corner_nm'] = '스포츠용품'
data.loc[data.corner_nm == '카세트,전화기', 'corner_nm'] = '전화기/카세트'

In [10]:
# pc_nm 에서 겹치는 값들 처리하기

data.loc[data.pc_nm == '침구,수예', 'pc_nm'] = '침구/수예'
data.loc[data.pc_nm == '디자이너부띠크', 'pc_nm'] = '디자이너부띠끄'
data.loc[data.pc_nm == '디자이너부틱', 'pc_nm'] = '디자이너부띠끄'
data.loc[data.pc_nm == '니트,단품,모피', 'pc_nm'] = '니트/단품/모피'
data.loc[data.pc_nm == '니트/단품', 'pc_nm'] = '니트/단품/모피'
data.loc[data.pc_nm == '로얄부틱', 'pc_nm'] = '로얄부띠끄'
data.loc[data.pc_nm == '트랜디 케쥬얼', 'pc_nm'] = '트랜디캐쥬얼'

In [11]:
# lgbm 에서 json 오류 방지를 위해 특수문자 , 제거

import re
data.rename(columns = lambda x:re.sub(',', '/', x), inplace = True)

# Categoric Feature Making

In [49]:
def aa(x):
    if len(x)>=1:
        return 1
    else :
        return 0

In [50]:
IDtest = test.custid.unique()

In [51]:
level = 'corner_nm'
data[level].nunique()

299

In [52]:
catFeatures_train_cor = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=aa, fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest').\
                         drop(columns=['custid']).values
catFeatures_test_cor = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=aa, fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values

# max_features = X_train_cat.shape[1]

In [54]:
catFeatures_train_cor = pd.DataFrame(catFeatures_train_cor)
catFeatures_test_cor = pd.DataFrame(catFeatures_test_cor)

In [55]:
catFeatures_train_cor.columns = catFeatures_train_cor.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )
catFeatures_test_cor.columns = catFeatures_test_cor.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )

**<font color='CC3D3D'>[brd_nm]**

In [57]:
level = 'brd_nm'
data[level].nunique()

1873

In [58]:
catFeatures_train_brd = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=aa, fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values

catFeatures_test_brd = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=aa, fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values
# max_features = X_train.shape[1]

In [59]:
catFeatures_train_brd = pd.DataFrame(catFeatures_train_brd)
catFeatures_test_brd = pd.DataFrame(catFeatures_test_brd)

In [60]:
catFeatures_train_brd.columns = catFeatures_train_brd.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )
catFeatures_test_brd.columns = catFeatures_test_brd.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )

**<font color='CC3D3D'>[pc_nm]**

In [61]:
level = 'pc_nm'
data[level].nunique()

71

In [62]:
catFeatures_train_pc = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=aa, fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values

catFeatures_test_pc = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=aa, fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values
# max_features = X_train.shape[1]

In [63]:
catFeatures_train_pc = pd.DataFrame(catFeatures_train_pc)
catFeatures_test_pc = pd.DataFrame(catFeatures_test_pc)

In [64]:
catFeatures_train_pc.columns = catFeatures_train_pc.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )
catFeatures_test_pc.columns = catFeatures_test_pc.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )

**<font color='CC3D3D'>[part_nm]**

In [65]:
level = 'part_nm'
data[level].nunique()

29

In [66]:
catFeatures_train_part = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=aa, fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values

catFeatures_test_part = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=aa, fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values
# max_features = X_train.shape[1]

In [67]:
catFeatures_train_part = pd.DataFrame(catFeatures_train_part)
catFeatures_test_part = pd.DataFrame(catFeatures_test_part)

In [68]:
catFeatures_train_part.columns = catFeatures_train_part.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )
catFeatures_test_part.columns = catFeatures_test_part.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )

**<font color='CC3D3D'>[team_nm]** 

In [70]:
level = 'team_nm'
data[level].nunique()

3

In [71]:
catFeatures_train_team = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=aa, fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                        drop(columns=['custid']).values

catFeatures_test_team = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=aa, fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                        drop(columns=['custid']).values
# max_features = X_train.shape[1]

In [72]:
catFeatures_train_team = pd.DataFrame(catFeatures_train_team)
catFeatures_test_team = pd.DataFrame(catFeatures_test_team)

In [73]:
catFeatures_train_team.columns = catFeatures_train_team.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )
catFeatures_test_team.columns = catFeatures_test_team.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )

**<font color='CC3D3D'>[buyer_nm]**

In [74]:
level = 'buyer_nm'
data[level].nunique()

34

In [75]:
catFeatures_train_buyer = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=aa, fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                        drop(columns=['custid']).values

catFeatures_test_buyer = pd.pivot_table(data, index='custid', columns=level, values='tot_amt',
                         aggfunc=aa, fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                        drop(columns=['custid']).values
# max_features = X_train.shape[1]

In [76]:
catFeatures_train_buyer = pd.DataFrame(catFeatures_train_buyer)
catFeatures_test_buyer = pd.DataFrame(catFeatures_test_buyer)

In [77]:
catFeatures_train_buyer.columns = catFeatures_train_buyer.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )
catFeatures_test_buyer.columns = catFeatures_test_buyer.columns.map(lambda x : level+ "_" + str(x) + "_onehot" )

# Merge Categoric Features

In [78]:
onehot_features_train = pd.concat([catFeatures_train_cor, catFeatures_train_brd, catFeatures_train_pc,\
                            catFeatures_train_part, catFeatures_train_team, catFeatures_train_buyer], axis=1)

In [79]:
onehot_features_test = pd.concat([catFeatures_test_cor, catFeatures_test_brd, catFeatures_test_pc, \
                           catFeatures_test_part, catFeatures_test_team, catFeatures_test_buyer], axis=1)

In [80]:
onehot_features_train.head()

,corner_nm_0_onehot,corner_nm_1_onehot,corner_nm_2_onehot,corner_nm_3_onehot,corner_nm_4_onehot,corner_nm_5_onehot,corner_nm_6_onehot,corner_nm_7_onehot,corner_nm_8_onehot,corner_nm_9_onehot,...,buyer_nm_24_onehot,buyer_nm_25_onehot,buyer_nm_26_onehot,buyer_nm_27_onehot,buyer_nm_28_onehot,buyer_nm_29_onehot,buyer_nm_30_onehot,buyer_nm_31_onehot,buyer_nm_32_onehot,buyer_nm_33_onehot
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


# Deployment

In [81]:
onehot_features_train.to_csv('hwang_onehot_features_train.csv', index=False)
onehot_features_test.to_csv('hwang_onehot_features_test.csv', index=False)